In [929]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from datetime import timedelta
import math 

In [930]:

# Cargar archivo Excel
file_path = "generic_input_case.xlsx"

# Leer cada hoja
df_horizonte = pd.read_excel(file_path, sheet_name="HORIZONTE")
df_bd_up = pd.read_excel(file_path, sheet_name="BD_UP")
df_frota = pd.read_excel(file_path, sheet_name="FROTA")
df_grua = pd.read_excel(file_path, sheet_name="GRUA")
df_fabrica = pd.read_excel(file_path, sheet_name="FABRICA")
df_rota = pd.read_excel(file_path, sheet_name="ROTA")



c:\Users\wilma\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [931]:

# --- Funciones de Validación de Restricciones ---

def validate_daily_demand(schedule_df, df_fabrica,flag):
    """Verifica si el volumen diario entregado está dentro de los límites."""
    daily_volume = schedule_df.groupby('DIA')['VOLUME'].sum()
    merged = pd.merge(daily_volume.reset_index(), df_fabrica[['DIA', 'DEMANDA_MIN', 'DEMANDA_MAX']], on='DIA')

    violations = merged[
        (merged['VOLUME'] < merged['DEMANDA_MIN']) |
        (merged['VOLUME'] > merged['DEMANDA_MAX'])
    ]

    if not violations.empty:
        if flag==1:
            print("Incumplimiento - Demanda Diaria:")
            print(violations)
        return False
    return True

def validate_daily_rsp(schedule_df, df_fabrica,flag):
    """Verifica si el RSP promedio ponderado diario está dentro de los límites."""
    if schedule_df.empty or 'VOLUME' not in schedule_df.columns or 'RSP' not in schedule_df.columns:
        return True # No hay datos para validar

    schedule_df['RSP_WEIGHTED_SUM'] = schedule_df['RSP'] * schedule_df['VOLUME']
    daily_rsp_sum = schedule_df.groupby('DIA')['RSP_WEIGHTED_SUM'].sum()
    daily_volume_sum = schedule_df.groupby('DIA')['VOLUME'].sum()

    # Evitar división por cero si un día no tiene volumen
    daily_avg_rsp = (daily_rsp_sum / daily_volume_sum).fillna(0).reset_index()
    daily_avg_rsp.rename(columns={0: 'RSP_AVG'}, inplace=True)

    merged = pd.merge(daily_avg_rsp, df_fabrica[['DIA', 'RSP_MIN', 'RSP_MAX']], on='DIA')

    # Añadir tolerancia pequeña para comparaciones de punto flotante
    tolerance = 1e-6
    violations = merged[
        (round(merged['RSP_AVG'],2) < round(merged['RSP_MIN'],2)) |
        (round(merged['RSP_AVG'],2) > round(merged['RSP_MAX'],2))
    ]
     # Limpiar columna auxiliar
    # schedule_df.drop(columns=['RSP_WEIGHTED_SUM'], inplace=True, errors='ignore')


    if not violations.empty:
        if flag==1:
            print("Incumplimiento - RSP Diario:")
            print(violations)
        return False
    return True

def validate_transporter_fleet_size(schedule_df, df_frota):
    """Verifica si la flota diaria por transportador está dentro de los límites."""
    # Asegúrate de contar vehículos únicos por transportador y día
    daily_fleet = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['QTD_VEICULOS'].sum().reset_index() # Suma todos los vehiculos asignados por transportador en el día. OJO: esto asume que QTD_VEICULOS es el total por asignación UP-Transp-Dia

    # Necesitamos comparar contra FROTA_MIN y FROTA_MAX por transportador
    # df_frota no tiene DIA, se aplica a todos los días
    merged = pd.merge(daily_fleet, df_frota[['TRANSPORTADOR', 'FROTA_MIN', 'FROTA_MAX']], on='TRANSPORTADOR')

    violations = merged[
        (merged['QTD_VEICULOS'] < merged['FROTA_MIN']) |
        (merged['QTD_VEICULOS'] > merged['FROTA_MAX'])
    ]

    if not violations.empty:
        print("Incumplimiento - Tamaño Flota Transportador:")
        print(violations)
        return False
    return True

def validate_crane_capacity(schedule_df, df_grua):
    """Verifica si un transportador atiende más UPs simultáneas que sus grúas."""
    # Contar UPs únicas atendidas por cada transportador cada día
    daily_up_count = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['UP'].nunique().reset_index()
    daily_up_count.rename(columns={'UP': 'UP_COUNT'}, inplace=True)

    merged = pd.merge(daily_up_count, df_grua[['TRANSPORTADOR', 'QTD_GRUAS']], on='TRANSPORTADOR')

    violations = merged[merged['UP_COUNT'] > merged['QTD_GRUAS']]

    if not violations.empty:
        print("Incumplimiento - Capacidad Grúas:")
        print(violations)
        return False
    return True

def validate_fazenda_exclusivity(schedule_df):
    """Verifica si un transportador opera en más de una Fazenda por día."""
    daily_fazenda_count = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['FAZENDA'].nunique().reset_index()
    daily_fazenda_count.rename(columns={'FAZENDA': 'FAZENDA_COUNT'}, inplace=True)

    violations = daily_fazenda_count[daily_fazenda_count['FAZENDA_COUNT'] > 1]

    if not violations.empty:
        print("Incumplimiento - Exclusividad Fazenda:")
        print(violations)
        return False
    return True

def validate_min_vehicles_per_up(schedule_df, df_grua):
    """Verifica el porcentaje mínimo de vehículos asignados a una UP."""
    # Calcular flota total por transportador por día
    total_daily_fleet = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['QTD_VEICULOS'].sum().reset_index()
    total_daily_fleet.rename(columns={'QTD_VEICULOS': 'TOTAL_VEHICULOS_DIA'}, inplace=True)

    # Unir con la asignación por UP y la info de grúa
    merged = pd.merge(schedule_df, total_daily_fleet, on=['DIA', 'TRANSPORTADOR'])
    merged = pd.merge(merged, df_grua[['TRANSPORTADOR', 'PORCENTAGEM_VEICULOS_MIN']], on='TRANSPORTADOR')

    # Calcular porcentaje real
    merged['PORCENTAJE_REAL'] = merged['QTD_VEICULOS'] / merged['TOTAL_VEHICULOS_DIA']

    # Añadir tolerancia pequeña
    tolerance = 1e-6
    violations = merged[merged['PORCENTAJE_REAL'] < merged['PORCENTAGEM_VEICULOS_MIN'] - tolerance]

    if not violations.empty:
        print("Incumplimiento - Vehículos Mínimos por UP:")
        # Mostrar solo columnas relevantes para la violación
        print(violations[['DIA', 'TRANSPORTADOR', 'UP', 'QTD_VEICULOS', 'TOTAL_VEHICULOS_DIA', 'PORCENTAGEM_VEICULOS_MIN', 'PORCENTAJE_REAL']])
        return False
    return True

# --- Funciones de Validación de Completitud (Más Complejas) ---
# Estas requieren analizar el historial a través de los días

def check_continuity(days_list):
    """Función auxiliar para verificar continuidad y número de bloques."""
    if not days_list:
        return 0, True # 0 bloques, continuo
    
    sorted_days = sorted(list(set(days_list)))
    if not sorted_days:
      return 0, True

    blocks = 0
    is_continuous = True
    if sorted_days:
        blocks = 1
        for i in range(len(sorted_days) - 1):
            if sorted_days[i+1] != sorted_days[i] + 1:
                blocks += 1
                is_continuous = False # Hay un salto
    return blocks, is_continuous

def validate_completion_rules(schedule_df, df_bd_up):
    """Valida las reglas de transporte completo/fraccionado para UPs y Fazendas."""
    all_valid = True
    total_volume_transported = schedule_df.groupby('UP')['VOLUME'].sum()
    
    # 1. Validación por UP
    for up_id, group in schedule_df.groupby('UP'):
        up_info = df_bd_up[df_bd_up['UP'] == up_id].iloc[0]
        total_volume_up = up_info['VOLUME']
        volume_transported_up = total_volume_transported.get(up_id, 0)
        days_worked = group['DIA'].tolist()
        num_blocks, is_continuous = check_continuity(days_worked)

        is_small_up = total_volume_up < 7000

        # Regla UP Pequeña (<7000 m³)
        if is_small_up:
            # Debe ser transportada completamente si se empezó
            if 0 < volume_transported_up < total_volume_up:
                 print(f"Incumplimiento UP < 7000: UP {up_id} iniciada pero no completada. Transportado: {volume_transported_up:.2f}, Total: {total_volume_up:.2f}")
                 all_valid = False
            # Debe tener solo 1 entrada (ser continua)
            if num_blocks > 1:
                 print(f"Incumplimiento UP < 7000: UP {up_id} tiene {num_blocks} entradas (debe ser 1). Días: {sorted(list(set(days_worked)))}")
                 all_valid = False

        # Regla UP Grande (>=7000 m³)
        else:
            # Puede tener hasta 2 entradas
            if num_blocks > 2:
                 print(f"Incumplimiento UP >= 7000: UP {up_id} tiene {num_blocks} entradas (máximo 2). Días: {sorted(list(set(days_worked)))}")
                 all_valid = False

    # # 2. Validación por Fazenda (Simplificada: verifica si fue completada si se inició)
    # # Una validación completa requeriría verificar si el transportador cambió *antes* de completar.
    # # Esta versión solo chequea si el volumen total fue transportado al final del horizonte.
    # fazenda_volume_transported = schedule_df.groupby('FAZENDA')['VOLUME'].sum()
    # # Calcular volumen total por Fazenda desde BD_UP
    # fazenda_total_volume = df_bd_up.groupby('FAZENDA')['VOLUME'].sum()

    # for fazenda, transported in fazenda_volume_transported.items():
    #     total = fazenda_total_volume.get(fazenda, 0)
    #     # Si se transportó algo pero no todo, es una violación potencial (simplificada)
    #     # La regla estricta es más compleja de verificar sin seguir estados diarios.
    #     if 0 < transported < total:
    #         # Esto es una indicación, no una prueba definitiva de violación de la regla *durante* la ejecución
    #         print(f"Advertencia - Completitud Fazenda: Fazenda {fazenda} iniciada pero no completada al final del horizonte. Transportado: {transported:.2f}, Total: {total:.2f}")
    #         # all_valid = False # Podrías marcarlo como inválido aquí si quieres ser estricto con el resultado final

    return all_valid





In [932]:

# --- Función Principal de Validación ---
def validate_essential_constraints(schedule_df, df_fabrica, df_frota, df_grua, df_bd_up,flag):
    # """Ejecuta todas las validaciones."""
    # print("--- Iniciando Validación de Restricciones ---")
    is_feasible = True

    # Limpiar columna auxiliar de RSP si existe de ejecuciones anteriores
    if 'RSP_WEIGHTED_SUM' in schedule_df.columns:
        schedule_df.drop(columns=['RSP_WEIGHTED_SUM'], inplace=True, errors='ignore')

    if not validate_daily_demand(schedule_df, df_fabrica,flag): is_feasible = False
    if not validate_daily_rsp(schedule_df, df_fabrica,flag): is_feasible = False
    if not validate_transporter_fleet_size(schedule_df, df_frota): is_feasible = False
    if not validate_crane_capacity(schedule_df, df_grua): is_feasible = False
    if not validate_fazenda_exclusivity(schedule_df): is_feasible = False
    
    return is_feasible

# --- Función Principal de Validación ---
def validate_all_constraints(schedule_df, df_fabrica, df_frota, df_grua, df_bd_up,flag):
    """Ejecuta todas las validaciones."""
    # print("--- Iniciando Validación de Restricciones ---")
    is_feasible = True

    # Limpiar columna auxiliar de RSP si existe de ejecuciones anteriores
    if 'RSP_WEIGHTED_SUM' in schedule_df.columns:
        schedule_df.drop(columns=['RSP_WEIGHTED_SUM'], inplace=True, errors='ignore')

    if not validate_daily_demand(schedule_df, df_fabrica,flag): is_feasible = False
    if not validate_daily_rsp(schedule_df, df_fabrica,flag): is_feasible = False
    if not validate_transporter_fleet_size(schedule_df, df_frota): is_feasible = False
    if not validate_crane_capacity(schedule_df, df_grua): is_feasible = False
    if not validate_fazenda_exclusivity(schedule_df): is_feasible = False
    # OJO: La validación de min_vehicles_per_up puede fallar si hay días/transportadores sin asignaciones en schedule_df
    # Se necesita asegurar que schedule_df contenga todas las asignaciones relevantes.
    # if not validate_min_vehicles_per_up(schedule_df, df_grua): is_feasible = False # Puede necesitar ajustes
    if not validate_completion_rules(schedule_df, df_bd_up): is_feasible = False

    print("--- Validación Terminada ---")
    if is_feasible:
        print(">>> La solución CUMPLE con todas las restricciones verificadas.")
    else:
        print(">>> La solución NO CUMPLE con una o más restricciones.")

    return is_feasible

In [933]:

# --- Funciones Auxiliares ---
def calculate_daily_transport_volume(vehicles, caixa_carga, tempo_ciclo):
    """Calcula el volumen máximo que pueden transportar los vehículos en un día."""
    if vehicles <= 0 or caixa_carga <= 0 or tempo_ciclo <= 0:
        return 0
    return vehicles * caixa_carga * tempo_ciclo

def check_continuity(days_list):
    """Función auxiliar para verificar continuidad y número de bloques."""
    if not isinstance(days_list, list) or not days_list:
         return 0, True
    
    sorted_days = sorted(list(set(filter(None, days_list)))) # Filtra None o NaNs
    if not sorted_days:
      return 0, True

    blocks = 0
    if sorted_days:
        blocks = 1
        for i in range(len(sorted_days) - 1):
            if sorted_days[i+1] != sorted_days[i] + 1:
                blocks += 1
    is_continuous = (blocks <= 1) # Consideramos continuo si hay 0 o 1 bloque
    return blocks, is_continuous


In [934]:
# --- Heurística Constructiva ---

def constructive_heuristic(df_horizonte, df_bd_up, df_frota, df_grua, df_fabrica, df_rota,flag):
    """
    Intenta construir una solución factible día por día.
    """
    schedule = [] # Lista para guardar las filas de la solución

    # --- Inicializar Estado ---
    up_status = df_bd_up.set_index('UP').copy() # Usar UP como índice para fácil acceso
    up_status['VOLUME_RESTANTE'] = up_status['VOLUME']
    # Inicializar DIAS_TRABAJADOS como una lista vacía para cada fila
    up_status['DIAS_TRABAJADOS'] = [[] for _ in range(len(up_status))]
    up_status['BLOQUES_TRABAJO'] = 0
    up_status['ESTADO'] = 'PENDIENTE' # PENDIENTE, ACTIVA, COMPLETADA, PAUSADA (>7000m3 only)

    # Estado del transportador por día (se resetea/actualiza cada día)
    transporter_daily_status = {t: {'fazenda_actual': None, 'ups_activas_hoy': [], 'vehiculos_usados_hoy': 0}
                                for t in df_frota['TRANSPORTADOR']}

    # Guardar la última Fazenda trabajada por un transportador para continuidad
    transporter_last_fazenda = {t: None for t in df_frota['TRANSPORTADOR']}
    transportadores=list(df_frota['TRANSPORTADOR'].unique())
    random.shuffle(transportadores)
    # print(up_status)
    # print("hi i'm here")
    up_status=up_status.sample(frac=1)
    # print(up_status)
    # print("--- Iniciando Heurística Constructiva ---")

    # --- Bucle Principal por Día ---
    for _, dia_info in df_horizonte.iterrows():
        dia = dia_info['DIA']
        mes = dia_info['MES']
        # print(f"\n--- Construyendo Día {dia} ---")

        # Resetear estado diario (mantener fazenda si es necesario?)
        for t in transporter_daily_status:
            transporter_daily_status[t]['ups_activas_hoy'] = []
            transporter_daily_status[t]['vehiculos_usados_hoy'] = 0
            # ¿Debería mantenerse la fazenda_actual si no se completó? Sí.
            # transporter_daily_status[t]['fazenda_actual'] = None # Resetear o mantener?

        # Obtener demandas y límites del día

        volumen_acumulado_dia = 0

        # --- Priorización de UPs ---
        # 1. UPs Activas o Pausadas (que deben continuarse)
        # 2. UPs Pendientes
        # Podría añadirse más lógica (ej. por RSP para cumplir objetivo diario)
        ups_activas_pausadas = up_status[up_status['ESTADO'].isin(['ACTIVA', 'PAUSADA'])].index.tolist()
        ups_pendientes = up_status[up_status['ESTADO'] == 'PENDIENTE'].index.tolist()
        # Simple Prio: Activas/Pausadas primero, luego Pendientes (podría ordenarse mejor)
        ups_priorizadas = ups_activas_pausadas + ups_pendientes

        # --- Bucle por Transportador ---
        # Podríamos ordenar transportadores (ej. por capacidad) pero lo hacemos simple
        
        for transportador in transportadores:
            
            transportador_info = df_frota[df_frota['TRANSPORTADOR'] == transportador].iloc[0]
            grua_info = df_grua[df_grua['TRANSPORTADOR'] == transportador].iloc[0]
            frota_min_transp = transportador_info['FROTA_MIN']
            frota_max_transp = transportador_info['FROTA_MAX']
            qtd_gruas_transp = grua_info['QTD_GRUAS']
            
            # --- Determinar Fazenda Objetivo ---
            fazenda_objetivo = None
            # Si el transportador estaba activo en una Fazenda ayer y no la completó, DEBE continuarla.
            fazenda_previa = transporter_last_fazenda[transportador]
            if fazenda_previa:
                 ups_en_fazenda_previa = up_status[up_status['FAZENDA'] == fazenda_previa]
                 # Si *alguna* UP en esa fazenda está ACTIVA o PAUSADA, debe continuarla.
                 if any(ups_en_fazenda_previa['ESTADO'].isin(['ACTIVA', 'PAUSADA'])):
                     fazenda_objetivo = fazenda_previa
                    #  print(f"  {transportador} debe continuar Fazenda: {fazenda_objetivo}")
            
            # Si no hay obligación de continuar, buscar una nueva Fazenda (o continuar una opcionalmente)
            if not fazenda_objetivo:
                 # Estrategia simple: Elegir la primera fazenda con UPs pendientes/activas que pueda atender
                 possible_fazendas = up_status[up_status.index.isin(ups_priorizadas)]['FAZENDA'].unique()
                 if (len(possible_fazendas)>0):
                     random.shuffle(possible_fazendas)
                #  print(possible_fazendas)
                 for f in possible_fazendas:
                     # Verificar si hay ruta para *alguna* UP en esa fazenda
                     ups_in_f = up_status[(up_status['FAZENDA'] == f) & (up_status.index.isin(ups_priorizadas))].index
                     if any(not df_rota[(df_rota['ORIGEM'] == up) & (df_rota['TRANSPORTADOR'] == transportador)].empty for up in ups_in_f):
                          fazenda_objetivo = f
                        #   print(f"  {transportador} elige nueva Fazenda: {fazenda_objetivo}")
                          break # Elegir la primera encontrada

            if not fazenda_objetivo:
                print(f"  {transportador} no encontró Fazenda objetivo hoy.")
                transporter_last_fazenda[transportador] = None # No trabajó
                continue # Siguiente transportador
                
            transporter_daily_status[transportador]['fazenda_actual'] = fazenda_objetivo
            transporter_last_fazenda[transportador] = fazenda_objetivo # Actualizar la última trabajada

            # --- Seleccionar UPs dentro de la Fazenda Objetivo ---
            ups_seleccionadas_hoy = []
            # Filtrar UPs priorizadas que estén en la fazenda objetivo
            ups_candidatas_fazenda = [up for up in ups_priorizadas if up_status.loc[up, 'FAZENDA'] == fazenda_objetivo]
            # print("FLAGGGGGGG", ups_candidatas_fazenda)
            for up_id in ups_candidatas_fazenda:
                if len(ups_seleccionadas_hoy) >= qtd_gruas_transp:
                    break # Límite de grúas alcanzado

                # Verificar si hay ruta
                rota_info = df_rota[(df_rota['ORIGEM'] == up_id) & (df_rota['TRANSPORTADOR'] == transportador)]
                if rota_info.empty:
                    continue

                # Verificar Reglas de Completitud/Fragmentación ANTES de añadir
                up_data = up_status.loc[up_id]
                vol_total = up_data['VOLUME']
                vol_restante = up_data['VOLUME_RESTANTE']
                estado_actual = up_data['ESTADO']
                bloques_actuales = up_data['BLOQUES_TRABAJO']

                if vol_restante <= 0: continue # Ya completada

                puede_empezar_o_continuar = False
                if estado_actual == 'PENDIENTE':
                    # Si es pequeña (<7000), solo puede empezar si se puede terminar hoy (simplificación: difícil saber a priori)
                    # Omitimos esa check compleja por ahora y permitimos empezar.
                    # Si es grande (>=7000), puede empezar si bloques < 2
                    if vol_total < 7000:
                         puede_empezar_o_continuar = True # Asumimos que puede empezar
                    elif bloques_actuales < 2:
                         puede_empezar_o_continuar = True
                elif estado_actual == 'ACTIVA':
                     puede_empezar_o_continuar = True # Puede continuar
                elif estado_actual == 'PAUSADA':
                     # Solo puede continuar si bloques < 2
                     if bloques_actuales < 2:
                          puede_empezar_o_continuar = True

                if puede_empezar_o_continuar:
                    ups_seleccionadas_hoy.append(up_id)

            if not ups_seleccionadas_hoy:
                print(f"  {transportador} no encontró UPs viables en {fazenda_objetivo} hoy.")
                continue

            # print(f"  {transportador} en {fazenda_objetivo} -> UPs: {ups_seleccionadas_hoy}")

            # --- Asignar Vehículos y Calcular Volumen ---
            num_ups_asignadas = len(ups_seleccionadas_hoy)
            vehiculos_total_asignar = 0

            # Intentar asignar al menos el mínimo de flota, distribuido
            if num_ups_asignadas > 0:
                 vehiculos_total_asignar = frota_max_transp#random.randint(frota_min_transp,frota_max_transp)

            distr=random.random()
            veh_up=[0 for _ in range (num_ups_asignadas)]
            ass_veh=vehiculos_total_asignar
            perc=grua_info["PORCENTAGEM_VEICULOS_MIN"]

            for i in range (num_ups_asignadas):
                if  (num_ups_asignadas==1):
                    veh_up[i]=int(vehiculos_total_asignar)
                    ass_veh-=veh_up[i]   
                elif distr< perc and i==0 and ass_veh>0:
                    veh_up[i]=math.ceil(perc*vehiculos_total_asignar)
                    ass_veh-=veh_up[i]
                elif distr > perc and distr<(1-perc) and i==0 and ass_veh>0:
                    veh_up[i]=math.ceil(distr*vehiculos_total_asignar)
                    ass_veh-=veh_up[i]   
                elif (distr > perc and i==0) and ass_veh>0:
                    veh_up[i]=int(vehiculos_total_asignar)
                    ass_veh-=veh_up[i]   
                   
                else:
                    veh_up[i]=min(math.ceil((1-perc)*vehiculos_total_asignar),ass_veh)
                    ass_veh-=max(veh_up[i],0)


            # Distribución simple (podría ser proporcional al volumen restante, etc.)
            vehiculos_por_up = {}
            if num_ups_asignadas > 0:
                
                for idx,up_id in enumerate(ups_seleccionadas_hoy):
                     vehi_up = veh_up[idx]

                     
                     # Verificar si con estos vehículos se puede transportar *algo*
                     rota_info = df_rota[(df_rota['ORIGEM'] == up_id) & (df_rota['TRANSPORTADOR'] == transportador)].iloc[0]
                     caixa = rota_info['CAIXA_CARGA']
                     ciclo = rota_info['TEMPO_CICLO'] # TODO: Ajustar por ciclo lento? (PDF no especifica cómo)
                     vol_potencial_up = calculate_daily_transport_volume(vehi_up, caixa, ciclo)
                     
                     if vol_potencial_up > 0:
                         vehiculos_por_up[up_id] = vehi_up

                         
                # Recalcular UPs activas y vehículos totales usados hoy
                ups_seleccionadas_hoy = list(vehiculos_por_up.keys())
                vehiculos_usados_transp_hoy = sum(vehiculos_por_up.values())
                
                # Validar flota total usada hoy
                if not (frota_min_transp <= vehiculos_usados_transp_hoy <= frota_max_transp):
                     print(f"Error interno: Flota calculada ({vehiculos_usados_transp_hoy}) fuera de rango [{frota_min_transp}, {frota_max_transp}] para {transportador}")
                     pass

                transporter_daily_status[transportador]['vehiculos_usados_hoy'] = vehiculos_usados_transp_hoy
                transporter_daily_status[transportador]['ups_activas_hoy'] = ups_seleccionadas_hoy

            # --- Calcular Volumen Transportado y Actualizar Estado ---
            for up_id in ups_seleccionadas_hoy:
                vehi_up = vehiculos_por_up[up_id]
                up_data = up_status.loc[up_id] # Obtener datos actuales
                rota_info = df_rota[(df_rota['ORIGEM'] == up_id) & (df_rota['TRANSPORTADOR'] == transportador)].iloc[0]
                caixa = rota_info['CAIXA_CARGA']
                ciclo = rota_info['TEMPO_CICLO'] # TODO: Ajustar ciclo lento
                
                vol_max_transportable_hoy = calculate_daily_transport_volume(vehi_up, caixa, ciclo)
                vol_real_transportado = min(vol_max_transportable_hoy, up_data['VOLUME_RESTANTE'])

                if vol_real_transportado > 0:
                    # Actualizar volumen restante
                    up_status.loc[up_id, 'VOLUME_RESTANTE'] -= vol_real_transportado

                    # --- Inicialización Segura de dias_previos ---
                    try:
                        # Intenta obtener la lista actual
                        retrieved_days = up_status.loc[up_id, 'DIAS_TRABAJADOS']
                        # Verifica si es una lista válida, si no, inicializa como vacía
                        if isinstance(retrieved_days, list):
                            dias_previos = retrieved_days.copy() # Usar .copy() por seguridad
                        else:
                            dias_previos = []
                    except KeyError:
                        # Error si up_id no existe (no debería pasar aquí, pero por seguridad)
                        print(f"Error Crítico: UP {up_id} no encontrado en up_status index.")
                        dias_previos = []
                    except Exception as e:
                            # Cualquier otro error inesperado al leer
                            print(f"Error inesperado al leer DIAS_TRABAJADOS para {up_id}: {e}")
                            dias_previos = []
                    # --- Fin Inicialización Segura ---

                    # Ahora estamos seguros de que 'dias_previos' es una lista

                    nuevo_estado = 'ACTIVA'

                    # Verificar si es un nuevo bloque de trabajo
                    # Un bloque nuevo empieza si la lista estaba vacía O si el día actual no es consecutivo al último registrado
                    is_new_block = False
                    if not dias_previos: # Es el primer día que se trabaja esta UP
                        is_new_block = True
                    elif dia != dias_previos[-1] + 1: # Hubo una interrupción (día no consecutivo)
                        is_new_block = True

                    # Incrementar contador de bloques SOLO si es un nuevo bloque Y NO es el primer día en absoluto
                    if is_new_block and dias_previos:
                        # Asegúrate de que BLOQUES_TRABAJO exista y sea numérico
                            if pd.isna(up_status.loc[up_id, 'BLOQUES_TRABAJO']):
                                up_status.loc[up_id, 'BLOQUES_TRABAJO'] = 0
                            up_status.loc[up_id, 'BLOQUES_TRABAJO'] += 1

                    # Añadir día actual a la lista si no está ya (por si acaso)
                    if dia not in dias_previos:
                        dias_previos.append(dia)

                    # Reasignar la lista actualizada usando .at para seguridad
                    up_status.at[up_id, 'DIAS_TRABAJADOS'] = dias_previos

                    # Actualizar estado final de la UP para hoy
                    if up_status.loc[up_id, 'VOLUME_RESTANTE'] <= 1e-6: # Usar tolerancia para flotantes
                        nuevo_estado = 'COMPLETADA'
                    # (Podrías añadir lógica para PAUSADA si es UP grande y se interrumpe, pero la dejaremos ACTIVA por ahora)
                    up_status.loc[up_id, 'ESTADO'] = nuevo_estado

                    # Añadir la entrada al schedule (DataFrame final)
                    schedule.append({
                        'UP': up_id,
                        'FAZENDA': up_data['FAZENDA'], # Necesitas obtener up_data antes en el loop
                        'TRANSPORTADOR': transportador,
                        'DIA': dia,
                        'MES': mes,
                        'DB': up_data['DB'],      # Necesitas obtener up_data antes en el loop
                        'RSP': up_data['RSP'],     # Necesitas obtener up_data antes en el loop
                        'QTD_VEICULOS': vehi_up,
                        'VOLUME': vol_real_transportado
                    })
                    volumen_acumulado_dia += vol_real_transportado
                    # print(f"    -> {transportador} transporta {vol_real_transportado:.2f} m³ desde {up_id} ({nuevo_estado})")

                    up_status.loc[up_id, 'ESTADO'] = nuevo_estado


    # --- Fin del Horizonte ---
    final_schedule_df = pd.DataFrame(schedule)

    # print("--- Heurística Constructiva Terminada ---")
    return final_schedule_df

In [935]:
df_horizonte1=df_horizonte#.iloc[0:10]
df_fabrica1=df_fabrica#.iloc[0:10]

flag=0
res=False
iter=0
while res==False and iter<10000:
    if iter % 50==0: print(f"Se está ejecutando la iteración {iter}")
    # print("Ejecutando heurística constructiva...")
    df_solucion_construida = constructive_heuristic(df_horizonte1, df_bd_up, df_frota, df_grua, df_fabrica1, df_rota,flag)
    # Validar la solución completa al final
    res=validate_essential_constraints(df_solucion_construida, df_fabrica1, df_frota, df_grua, df_bd_up,flag)
    iter+=1
if res==True:
    print(f"after {iter} iterations a feasible solution was found!!")
    print(df_solucion_construida)
else:
    print("No feasible solution was found :(")

Se está ejecutando la iteración 0
  Rampazo no encontró UPs viables en PIRACEMA_GLEBA PULADOR - DX hoy.
  Rampazo no encontró UPs viables en TURVO III (LEX) hoy.
  Rampazo no encontró UPs viables en PIRACEMA_GLEBA PULADOR - DX hoy.
  Rampazo no encontró UPs viables en TURVO III (LEX) hoy.
  Pastori no encontró UPs viables en PIRACEMA_GLEBA PULADOR - DX hoy.
  Rampazo no encontró UPs viables en PIRACEMA_GLEBA PULADOR - DX hoy.
  Rampazo no encontró UPs viables en PIRACEMA_GLEBA PULADOR - DX hoy.
  Pastori no encontró UPs viables en TURVO III (LEX) hoy.
  Rampazo no encontró UPs viables en PIRACEMA_GLEBA PULADOR - DX hoy.
  Rampazo no encontró UPs viables en TURVO III (LEX) hoy.
  Pastori no encontró UPs viables en TURVO III (LEX) hoy.
Se está ejecutando la iteración 50
  Pastori no encontró UPs viables en TURVO III (LEX) hoy.
  Pastori no encontró UPs viables en PIRACEMA_GLEBA PULADOR - DX hoy.
  Pastori no encontró UPs viables en PIRACEMA_GLEBA PULADOR - DX hoy.
  Rampazo no encontró U

KeyboardInterrupt: 

In [ ]:
print(len(df_solucion_construida))
print(validate_all_constraints(df_solucion_construida, df_fabrica1, df_frota, df_grua, df_bd_up,flag))

47
Incumplimiento UP < 7000: UP S3AX06 iniciada pero no completada. Transportado: 2851.20, Total: 5568.94
Incumplimiento UP < 7000: UP S6C297 iniciada pero no completada. Transportado: 1108.80, Total: 4791.00
Incumplimiento UP < 7000: UP S6C334 iniciada pero no completada. Transportado: 5974.00, Total: 5974.00
Incumplimiento UP < 7000: UP S6C334 tiene 2 entradas (debe ser 1). Días: [1, 3, 4, 5, 6, 7]
--- Validación Terminada ---
>>> La solución NO CUMPLE con una o más restricciones.
False
